In [101]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes
import py_vollib_vectorized

Code to store the spot, expiration grids for pricing

In [102]:
class grid_params:
    def __init__(self, num_time_steps,low_spot, spot_incr, num_spots,max_texp) -> None:
        self.num_time_steps = num_time_steps
        self.low_spot = low_spot
        self.spot_incr = spot_incr
        self.num_spots = num_spots
        self.max_texp=max_texp
        self.texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
        self.spot_dim=np.arange(low_spot,low_spot+num_spots*spot_incr,spot_incr)
        self.time_steps = np.arange(0, num_time_steps, 1)
        self.num_spots=len(self.spot_dim)
        [self.spot_grid,self.texp_grid]=np.meshgrid(self.spot_dim,self.texp_dim)

class market_data:
    def __init__(self, spot, q, r, vol,slope) -> None:
        self.spot = spot
        self.q = q
        self.r = r
        self.vol = vol
        self.slope=slope
    

input texp grid goes from 1 to max_texp.
For each instrument I need to adjust the grid to go from texp for the instrument to zero and then stay at zero

In [103]:
def create_adjusted_texp_grid(texp_grid, option_texp, max_texp):
    texp_gridi=texp_grid-(max_texp-option_texp)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    return texp_gridi_zero

def create_constant_grid(param, num_time_steps, num_spots):
    cp_grid=np.full((num_time_steps,num_spots),param, dtype=type(param))
    return cp_grid

Initialize all of the data structures so that everything works properly when test step by step

In [104]:
num_time_steps=10
num_spots=19
low_strike=.5
incr_strike=.1
high_strike=2.0

In [145]:
qty=np.array([-1,1]*2)*1000000
texp=[1,1,2,2]
k=[.9,1.2]*2
k_dlr=k
cp=['p','c']*2
sigma=[.20,.2]*2

md=market_data(100,0,0,.2,.1)
data = {'qty': qty,
        'texp': texp,
        'k':k,
        '$k':k,
        'cp': cp,
        'sigma':sigma}
df_inp=pd.DataFrame(data)
max_texp=np.max(texp)
grid_settings=grid_params(num_time_steps,low_strike,incr_strike,num_spots,max_texp)
#vars(grid_settings)

In [106]:
def adjust_params_scenario(row, inplace=False):
    spot_jump=-.25
    result=row
    if(inplace==False):
        result=row.copy()
    #result=row.copy()
    result["spot"]=row["spot"]*(1+spot_jump)
    return result

In [147]:
def create_input_grids(r, q,spot_price, df_inp,grid_settings):
    df_grids=df_inp.copy()

    df_grids["r"]=r
    df_grids["q"]=q
    df_grids["texp"]=df_grids["texp"].apply(lambda x: create_adjusted_texp_grid(grid_settings.texp_grid, x, max_texp))
    df_grids[["qty","k","cp","sigma"]]=df_inp[["qty","k","cp","sigma"]].applymap(lambda x: create_constant_grid(x, grid_settings.num_time_steps, grid_settings.num_spots))
    df_grids[["q","r"]]=df_grids[["q","r"]].applymap(lambda x: create_constant_grid(x, grid_settings.num_time_steps, grid_settings.num_spots))
    df_grids[["spot"]]=df_inp[["qty"]].applymap(lambda x: grid_settings.spot_grid)
    bs_input={}

    for col in df_grids.columns:
        bs_input[col]=np.stack(list(df_grids[col]))
    
    return (df_grids, bs_input)

(df_grids, bs_input)=create_input_grids(.01,0,1,df_inp,grid_settings)

bs_input["spot"].shape

(4, 10, 20)

In [149]:

def price_grids(opt,mkt_data):
    #1 compute the junmp price
    #2 compute jump P&L
    #3 add the option price and jump p&L into the greeks dictionary
    #4 reshape the greeks dictionary
    #5 compute the totals
    #6 return the greeks dictionary
    #for some reason the vollib vectorized functions only work with flattened CP arrays
    #they also return a flattened array output so we need to reshape the output
    #to match the input shape
    
    option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(opt['cp'].flatten(), opt["spot"], opt['k'],opt['texp'], mkt_data.r, opt['sigma'],mkt_data.q,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(opt['cp'].flatten(), opt["spot"], opt['k'], opt['texp'], mkt_data.r, opt["sigma"],mkt_data.q,model='black_scholes_merton', return_as='dict')

    opt_jump=opt.copy()
    adjust_params_scenario(opt_jump,inplace=True)
    jump_option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(opt_jump['cp'].flatten(), opt_jump["spot"], opt_jump['k'],opt_jump['texp'], mkt_data.r, opt_jump['sigma'],mkt_data.q,return_as='array')

    option_greeks["price"]=option_price
    option_greeks["price_jump"]=jump_option_price
    option_greeks = {k: np.array(v).reshape(opt['k'].shape) for k, v in option_greeks.items()}
    option_greeks = {k: np.where(opt["texp"]>0,v,0) for k, v in option_greeks.items()}
    option_greeks["jump_pl"]=(option_greeks["price_jump"]-option_greeks["price"])-option_greeks["delta"]*(opt_jump["spot"]-opt["spot"])
    option_greeks["gamma"]=option_greeks["gamma"]/100

    dict_sum_grids={}
    for field in ["price","delta","gamma","vega","rho","theta","jump_pl","price_jump"]:
        dict_sum_grids[field]=(option_greeks[field]*opt["qty"]).sum(axis=0)
    return(option_greeks,dict_sum_grids)


option_greeks,sum_greeks=price_grids(bs_input,md)
option_greeks["price"].shape 

(4, 10, 20)

In [150]:
def price_input_df(df,r,q):
    option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(df['cp'], 1, df['k'],df['texp'], r, df['sigma'],q,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(df['cp'], 1, df['k'], df['texp'], r, df["sigma"],q,model='black_scholes_merton', return_as='dict')
    df['cp']
    df["price"]=option_price
    df["delta"]=option_greeks["delta"]
    df["gamma"]=option_greeks["gamma"]
    df["vega"]=option_greeks["vega"]

df_priced=df_inp.copy()
price_input_df(df_priced, md.r, md.q)
df_priced
vars(md)

{'spot': 100, 'q': 0, 'r': 0, 'vol': 0.2, 'slope': 0.1}

In [143]:
import ipywidgets as widgets
from IPython.display import display

w_greek_selector=widgets.Dropdown(
    options=list(sum_greeks.keys()),
    value=list(sum_greeks.keys())[3],
    #rows=10,
    description='Columns to graph',
    disabled=False
)

w_option_pricing=widgets.Output()

w_box=widgets.VBox([w_greek_selector,w_option_pricing])

def display_field(field):
    (df_grids,bs_input)=create_input_grids(md.r,md.q, 1, df_inp, grid_settings)
    (dict_grids,dict_sum_grids)=price_grids(bs_input,md)
    scales_factor={'price':1,'delta':1,'gamma':1/1000000*md.spot,'rho':md.spot,'theta':1,'jump_pl':md.spot/1000000,'vega':md.spot/1000,'price_jump':1}
    df = pd.DataFrame(dict_sum_grids[field]*scales_factor[field], columns = grid_settings.spot_dim, index = grid_settings.texp_dim)
    display(df.style.format('{:.1f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2))
    df_priced=df_inp.copy()
    price_input_df(df_priced, md.r, md.q)
    df_priced["price"]*=md.spot
    df_priced["$k"]=df_priced["k"]*md.spot
    w_option_pricing.clear_output()
    with w_option_pricing:
        
        display(df_priced.style.format({"qty": "{:,d}", 
                          "texp": "{:.2f}", 
                          "price": "{:.2f}",
                          "k": "{:.2f}",
                          "$k": "${:.2f}",
                          "sigma": "{:.2%}", 
                          "delta":"{:.2%}"}))
out1 = widgets.interactive_output(display_field, {'field': w_greek_selector})
display(w_box)
display(out1)


Output()

In [152]:
#inputs go here

qty=np.array([-1,1]*2)*1000000
texp=[1,1,2,2]
k=[.9,1.2]*2
k_dlr=k
cp=['p','c']*2
sigma=[.20,.2]*2
r=0
q=0
spot=100


######## don't change below this line: this is putting the inputs into a dataframe
md=market_data(spot,r,q,.2,.1)
data = {'qty': qty,
        'texp': texp,
        'k':k,
        '$k':k,
        'cp': cp,
        'sigma':sigma}
df_inp=pd.DataFrame(data)
max_texp=np.max(texp)
grid_settings=grid_params(num_time_steps,low_strike,incr_strike,num_spots,max_texp)
vars(grid_settings)
display(w_box)
display(out1)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<pandas.io.formats.style.Styler at 0x2…